The [lvlspy API](https://pypi.org/project/lvlspy/) was developed to create an easy-to-use tool set that can handle nuclear level data and facilitate calculations. Tutorials on how to fully utilize the API can be found [here](https://github.com/jaadt7/lvlspy_tutorial). This notebook will utilize the API to calculate the effective transition rate between the isomeric and ground state of $^{26}\mathrm{Al}$. The calculation itself is detailed in the paper by [Gupta and Meyer](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.64.025805).

To start things off, we check and (quietly) install any missing required python packages and import them.

In [ ]:
import sys, subprocess,pkg_resources
required = {'numpy','matplotlib','lvlspy','ipython','tabulate'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])

import tabulate
import numpy as np
import lvlspy.spcoll as lc
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import HTML, display, Math,Markdown

We define the generalized functions according to the methods defined in the aforementioned paper. The transfer properties function takes the rate matrix as input and extracts the Destruction Matrix diagonal, and sets up the Transfer matrix and the Production vectors. In the paper, the 'ij' subscripts refer to transitions from level 'i' into level 'j'. This however does not correspond to the 'ith' row and 'jth' column of the matrix. The index swap in the function takes care of this change.

In the lambda_effective function, the effective decay rate and the excitation rate are calculated between the ground and isomeric state. However, this notebook deviates from the method described in the paper by ignoring the approximation surrounding $(I - F^{T})^{-1}$ and calculates it directly. This leads to a change of form for $\lambda_{12}^{eff}$ and $\lambda_{21}^{eff}$ to $\Lambda_{1}(1 - (f_{1}^{in})^{T}(I - F^{T})^{-1})f_{1}^{out}$ and $\Lambda_{2}(f_{1}^{in})^{T}(I - F^{T})^{-1}f_{2}^{out}$ respectively.

The gamma_calculator function calculates $\Gamma_{q}^{in}$ and $\Gamma_{q}^{out}$ using the transfer properties.

The f's and Newton-Raphson functions are related to another and are used to solve coupled systems.

In [ ]:
# this function calculates the transfer properties required to calculate the various 
# properties this notebook will pace through########################################
def transfer_properties(rate_matrix):
    
    n = len(rate_matrix)
    
    lambda_1_in = rate_matrix[0,2:n]
    lambda_2_in = rate_matrix[1,2:n]

    lambda_1_out = rate_matrix[2:n,0]
    lambda_2_out = rate_matrix[2:n,1]

    LAMBDA = np.diag(rate_matrix)
    
    f1_out = lambda_1_out/LAMBDA[0]
    f2_out = lambda_2_out/LAMBDA[1]

    f1_in = lambda_1_in/LAMBDA[2:n]
    f2_in = lambda_2_in/LAMBDA[2:n]

    F = rate_matrix.T
    
    F = F[2:n,2:n]
    F = F/LAMBDA[2:n,None] #This only works if the arrays are numpy arrays
    
    np.fill_diagonal(F,0.0)
  
    return F,f1_in,f1_out,f2_in,f2_out,LAMBDA
########################################################################################################
#This function calculates the effective transition rates in and out between the ground and isomer states
def lambda_effective(T,sp):

    rate_matrix = np.abs(sp.compute_rate_matrix(T))
    F,f1_in,f1_out,f2_in,f2_out,LAMBDA = transfer_properties(rate_matrix)
    n = len(F)
    #Lambda_21_eff
    l_21 = LAMBDA[1]*np.matmul(f1_in,np.matmul(np.linalg.inv(np.identity(n) - F.T),f2_out))
    #Lambda_12_eff
    l_12 = LAMBDA[0]*(1.0 - np.matmul(f1_in,np.matmul(np.linalg.inv(np.identity(n) - F.T),f1_out)))
            
    return l_12,l_21
##############################################################################################################
#This function calculates the cascade probabilites in and out of the groud state and isomer
def gamma_calculator(T,sp):
    rate_matrix = np.abs(sp.compute_rate_matrix(T))
    F,f1_in,f1_out,f2_in,f2_out,LAMBDA = transfer_properties(rate_matrix)

    n = len(F)

    #gamma in
    g1_in = np.matmul(np.linalg.inv(np.identity(n) - F),f1_in)
    g2_in = np.matmul(np.linalg.inv(np.identity(n) - F),f2_in)

    #gamma out
    g1_out = np.matmul(np.linalg.inv(np.identity(n) - F.T),f1_out)
    g2_out = np.matmul(np.linalg.inv(np.identity(n) - F.T),f2_out)

    return g1_out,g1_in,g2_out,g2_in
##############################################################################################
# This block is responsible for solving the 2 level system####################################
def f1(Y_guess,Y_old,dt,l12,l21):
    return Y_guess[0]*(dt*l12 + 1.0) - dt*l21*Y_guess[1] - Y_old[0]

def f2(Y_guess,Y_old,dt,l12,l21):
    return Y_guess[1]*(1.0 + dt*l21) - dt*l12*Y_guess[0] - Y_old[1]

def newton_raphson(t,tol,Y,l12,l21):
    Y_guess = Y_old   = Y
    
    n = len(t)
    
    delta = np.ones(2)

    for i in range(1,n):
        dt = t[i] - t[i-1]
        #matrix A changes with each dt
        A = np.array([[1.0 + dt*l12,-dt*l21],[-dt*l12,1.0 + dt*l21]])
        while (max(abs(delta)) >= tol):
            b = np.array([-f1(Y_guess,Y_old,dt,l12,l21),-f2(Y_guess,Y_old,dt,l12,l21)])
            delta = np.linalg.solve(A,b)
            Y_guess += delta            
        
        delta = np.ones(2) #reset the delta's to make sure the while loop runs a calculation
        Y_old = Y_guess
        Y = np.vstack((Y,Y_old))
        
    return Y
#################################################################################################
# This definition is responsible for calculating the weights used to solve the ensemble system
def ensemble_weights(eq_prob,g1_in,g2_in):
    n = len(eq_prob)
    w_1,w_2,R_1k,R_2k = np.empty(n),np.empty(n),np.empty(n-2),np.empty(n-2)
    for i in range(n-2):
        R_1k[i] = eq_prob[i+2]/eq_prob[0] 
        R_2k[i] = eq_prob[i+2]/eq_prob[1]
    
    for i in range(n):
        if i == 0:
            w_1[i] = 1.0
            w_2[i] = 0.0
        elif i == 1:
            w_1[i] = 0.0
            w_2[i] = 1.0
        else:
            w_1[i] = g1_in[i-2]*R_1k[i-2]
            w_2[i] = g2_in[i-2]*R_2k[i-2]      


    W_1,W_2 = np.sum(w_1),np.sum(w_2)
    #partition functions for ground and isomer
    G1 = 11*W_1 #ground state has multiplicity 11 (2J + 1)
    G2 = W_2    #isomer has multiplicity 1 (2J +1)
    return w_1,w_2,W_1,W_2,R_1k,R_2k,G1,G2
############################################################################################
##### THis block is the of functions related to solving the whole reaction network
def f_vector(y_dt,y_i,A):
    return np.matmul(A,y_dt) - y_i

def nefraf_matrix(t,n_rows,tol,y0,A):
    y_result = np.empty([n_rows,len(t)])
    y_result[:,0] = y0

    I = np.identity(n_rows)
    for i in range(1,len(t)):
        delta = np.ones(n_rows)
        y_guess = y_old = y_result[:,i-1]
        dt = t[i] - t[i-1]
        mat_A = I - dt*A
        while max(abs(delta)) > tol:
           delta = np.linalg.solve(mat_A,-f_vector(y_guess,y_old,mat_A)) 
           y_guess += delta
        
        y_result[:,i] = y_guess
        
    return y_result
####################################################################################

Here we import our XML containing the level data pertaining to $^{26}\mathrm{Al}$. A tutorial to create your own XML with a different species and levels can be found [here](https://github.com/jaadt7/lvlspy_tutorial). If you wish to use it with this notebook, simply place the file in the same directory as the notebook and replace the XML name and the species name you are studying.

In [ ]:
#setup the collection array to read the xml and load the al26 data
new_coll = lc.SpColl()
new_coll.update_from_xml('al26_22_level.xml')#loading the XML

sp = new_coll.get()['al26'] #species name

#Temperature Range in K
T = np.logspace(8,10)

#initializing the rate arrays as a function of temperature
lambda_21_eff = np.empty(len(T))
lambda_12_eff = np.empty(len(T))

#Calculating the effective transition rates
for i,t in enumerate(T):
    lambda_12_eff[i],lambda_21_eff[i] = lambda_effective(T[i],sp)

Here we graph the results. We see that the graph follows the trend in Fig. 1 of the paper quiet well. The major difference here is for $T_{9} > 1.1$, the rate isn't as high. This is attributed to the fact that the XML being used has only 22 energy levels while the paper's calculation used 67. This matters since the effective rate is calculated via indirect transitions from upper lying states. The less higher-end states you have, the less contribution you will have at higher temperatures since they mainly populated under said condition.

In [ ]:
fontsize = 18

plt.figure(figsize = (9,9))
plt.rcParams['font.size'] = fontsize
plt.xscale('log')
plt.yscale('log')

plt.ylim([1.e-15,1.e+15])
plt.xlim([np.min(T)/1e+9,np.max(T)/1e+9])

plt.plot(T/1e+9,lambda_21_eff,color = 'red',linewidth = 2,label = 'Full Calculation')

plt.ylabel(r'$\lambda_{21}^{eff} (s^{-1})$')
plt.xlabel(r'$T_{9}$')

plt.yticks([1e-15,1e-10,1e-5,1e0,1e+5,1e+10,1e+15])

plt.axhline(y = 1.e-1,ls = '--',color = 'black',label = r'$0^{+}$ state $\beta^{-}$ Decay Rate')

plt.legend()
plt.show()

The effective rate could be understood via a combinatorial interpretation. The expression $(f_{1}^{in})^{T}(I - F^{T})^{-1}f_{2}^{out}$ from $\lambda_{21}^{eff}$ is actually the effective branching ratio in terms of Graph Theory. The "cascade probability vectors" are defined as $\Gamma_{q}^{in} = (I - F)^{-1}f_{q}^{in}$. The term cascade refers to the de-excitation from higher states into state 'q'. The paper also defines the 'infinite'-arc generalization of $f_{q}^{out}$ with $\Gamma_{q}^{out} = (I - F^{T})^{-1}f_{q}^{out}$. They then go through the proof to rewrite $\lambda_{21}^{eff} = (\lambda_{2}^{out})^{T}\Gamma_{1}^{in}$.

Other than cleaning up the expression for the effective rates, the $\Gamma$'s play another role, that of calculating the fugacity of each state. Fugacity here deviates away from the pressure definition it uses in thermodynamics, but the term was appropriated due to its description of a state's abundance likeliness to jump to another state with less fugacity. If two states have the same fugacity, they will not exchange abundances. Equilibrium is established when all states have the same fugacity of one.

In [ ]:
#Temperature in Kelvin
T = 2e+9

#calculate gamma's

g1_out,g1_in,g2_out,g2_in = gamma_calculator(T,sp)

#time array
n_time = 300
t = np.logspace(-15,-8,n_time)

#equilibrium abundances. The API calculates them for the whole system. For this calculation
#we will use only that for the ground and isomeric state
eq_prob = sp.compute_equilibrium_probabilities(T)

y1_eq = eq_prob[0]
y2_eq = eq_prob[1]

w_1,w_2,W_1,W_2,R_1k,R_2k,G1,G2 = ensemble_weights(eq_prob,g1_in,g2_in)

#initial ensemble abundances. System setup such that the system starts in the ground state.
Y = np.array([1.0,0.0])

#toleranace for Newton-Raphson convergence
tol = 1e-6

#calculating the effective transition rates for the ensemble calculation
l12,l21 = lambda_effective(T,sp)

l12,l21 = l12/W_1,l21/W_2

#evolving the system
Y = newton_raphson(t,tol,Y,l12,l21)
y1eq = W_1*eq_prob[0]
y2eq = W_2*eq_prob[1]

#setting up fugacity matrix as function of time
n_rows = len(eq_prob)

y = np.empty([n_rows,n_time])
y[0,:] = Y[:,0]/W_1
y[1,:] = Y[:,1]/W_2
PHI = np.empty([n_rows,n_time])

PHI[0,:], PHI[1,:] = y[0,:]/y1_eq, y[1,:]/y2_eq

#calculating fugacities based on combination of phi1 and phi2 (eqn 5.8 in paper)
for i in range(n_time):
    PHI[2:n_rows,i] = g1_in*PHI[0,i] + g2_in*PHI[1,i]

In [ ]:
plt.figure(figsize = (12,12))
plt.rcParams['font.size'] = fontsize

plt.scatter(t,Y[:,0],label = r'$Y_{(1)}$')
plt.scatter(t,Y[:,1],label = r'$Y_{(2)}$')
plt.scatter(t[-1],y1eq,marker ='*',color = 'red',label = r'$Y_{(1)}^{eq}$')
plt.scatter(t[-1],y2eq,marker ='*',color = 'black',label = r'$Y_{(2)}^{eq}$')


plt.xscale('log')

plt.legend()
plt.show()

Here we sample over temperatures chosen in table 4 from the paper to compare the values of $W_{1}(T)$, $W_{2}(T)$, $G_{1}(T)$, and $G_{2}(T)$, as well as $\lambda_{21}^{eff}$. By comparing the values to those in the paper, there is good agreement in between the factors listed above for values of $T_{9}$ up to 1.0. The deviation with higher values of and about $T_{9}$ are due to the fact that the xml used contains only 22 levels while the paper's calculation is using 67

In [ ]:
T9 = np.array([0.0100,0.0125,0.0150,0.0175,0.0200,0.0250,0.0300,0.0400,0.0500,0.0600,0.0700,0.0800,0.0900,0.1,
               0.125,0.15,0.175,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.25,1.5,1.75,2.0,2.5,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0])
n = len(T9)

table = np.empty((n,6))

for i in range(n):
    table[i,0],t9 = T9[i], T9[i]*1e+9
    g1_out,g1_in,g2_out,g2_in = gamma_calculator(t9,sp)
    eqprob = sp.compute_equilibrium_probabilities(t9)
    w_1,w_2,table[i,2],table[i,3],R_1k,R_2k,table[i,4],table[i,5] = ensemble_weights(eqprob,g1_in,g2_in)
    l12,table[i,1] = lambda_effective(t9,sp)

display(Math(r'\;\;\;\;\;\;T_{9} \;\;\;\;\;\;\;\;\; \lambda_{21}^{eff} (s^{-1})\;\;\;\;\;  W_{1}(T)\;\;\;\;\;\;\;\; W_{2}(T)\;\;\;\;\;\;\; G_{1}(T)\;\;\;\;\;\;\;G_{2}(T) '))
display(HTML(tabulate.tabulate(table,tablefmt='html',floatfmt=('.4f','.5e','.6f','.6f','.5f','.6f'))))


Here we will animate the evolution of the fugacities for the 22 levels in our xml calculated via the ensemble method

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.rcParams['font.size'] = fontsize
#extract energies from the levels in the xml
levs = sp.get_levels()
E = np.empty(n_rows)
for i in range(n_rows):
    E[i] = levs[i].get_energy()

def updatefig(i):
    fig.clear()
    fig.suptitle(r'time (s): %8.2e; $T_{9}$ = %4.2f' %(t[i],T/1e+9))
    plt.scatter(PHI[0,i],E[0],color = 'green') #ground state
    plt.scatter(PHI[1,i],E[1],color = 'red') #isomer
    plt.scatter(PHI[2:n_rows-1,i],E[2:len(E)-1],color = 'black')
    plt.axvline(x = PHI[0,i],color = 'green')
    plt.axvline(x = PHI[1,i],color = 'red')
    plt.xlabel(r'$\phi$')
    plt.ylabel('E (keV)')
    plt.xscale('log')
    plt.xlim([1e-8,10])
    plt.draw()

anim = animation.FuncAnimation(fig,updatefig,n_time)
display(HTML(anim.to_jshtml()))
plt.close() #closes the last frame

Here we perform the full network calculation of the 22 levels

In [ ]:
A = sp.compute_rate_matrix(T)
Y_0 = np.zeros(n_rows)
Y_0[0] = 1.0

Y_result = nefraf_matrix(t,n_rows,tol,Y_0,A)

fugacities = np.empty((Y_result.shape[0], Y_result.shape[1]))

for i in range(n_rows):
    fugacities[i, :] = Y_result[i, :] / eq_prob[i]

fig_1 = plt.figure(figsize = (10,10))
plt.rcParams['font.size'] = fontsize

def updatefig_2(i):
    fig_1.clear()
    fig_1.suptitle(r'time (s): %8.2e; $T_{9}$ = %4.2f' %(t[i],T/1e+9))
    plt.scatter(fugacities[0,i],E[0],color = 'green') #ground state
    plt.scatter(fugacities[1,i],E[1],color = 'red') #isomer
    plt.scatter(fugacities[2:n_rows-1,i],E[2:len(E)-1],color = 'black')
    plt.axvline(x = fugacities[0,i],color = 'green')
    plt.axvline(x = fugacities[1,i],color = 'red')
    plt.xlabel(r'$\phi$')
    plt.ylabel('E (keV)')
    plt.xscale('log')
    plt.xlim([1e-15,1e+2])
    plt.draw()

anim = animation.FuncAnimation(fig_1,updatefig_2,n_time)
display(HTML(anim.to_jshtml()))
plt.close() #closes the last frame


Here we overlay both animations to compare

In [ ]:
fig_2 = plt.figure(figsize = (10,10))
plt.rcParams['font.size'] = fontsize

def updatefig_3(i):
    fig_2.clear()
    fig_2.suptitle(r'time (s): %8.2e; $T_{9}$ = %4.2f' %(t[i],T/1e+9))
    plt.scatter(fugacities[0,i],E[0],color = 'green',) #ground state
    plt.scatter(fugacities[1,i],E[1],color = 'red') #isomer
    plt.scatter(fugacities[2:n_rows-1,i],E[2:len(E)-1],color = 'black')
    plt.axvline(x = fugacities[0,i],color = 'green')
    plt.axvline(x = fugacities[1,i],color = 'red')

    #empty circles indicate the ensemble calculation###################
    plt.scatter(PHI[0,i],E[0],facecolors = 'none',color = 'green') #ground state
    plt.scatter(PHI[1,i],E[1],facecolors = 'none',color = 'red') #isomer
    plt.scatter(PHI[2:n_rows-1,i],E[2:len(E)-1],facecolors = 'none',color = 'black')
    plt.axvline(x = PHI[0,i],color = 'green')
    plt.axvline(x = PHI[1,i],color = 'red')
    ######################################################################



    plt.xlabel(r'$\phi$')
    plt.ylabel('E (keV)')
    plt.xscale('log')
    plt.xlim([1e-15,1e+2])
    plt.draw()

anim = animation.FuncAnimation(fig_2,updatefig_3,n_time)
display(HTML(anim.to_jshtml()))
plt.close() #closes the last frame